# SFT Data Verification

This notebook loads and examines the SFT training data to verify the prompt and label structure used in the training script.

In [1]:
import os
import torch
from datasets import load_from_disk, DatasetDict, concatenate_datasets
from transformers import AutoTokenizer
from torch.utils.data import Dataset
import sys

# Add the path to import the SFT training modules
sys.path.append('/home/hice1/qfitterey3/scratch/LLada-Reasoning/sft_training')

print("Libraries imported successfully")

/home/hice1/qfitterey3/scratch/envs/llada/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Libraries imported successfully


In [2]:
# Recreate the SFTDataset class from the training script
class SFTDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.pad_id = tokenizer.pad_token_id

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        ex = self.dataset[idx]
        user_ids = self.tokenizer.encode(ex['user'] + self.tokenizer.eos_token, add_special_tokens=False)
        asm_ids = self.tokenizer.encode(ex['assistant'] + self.tokenizer.eos_token, add_special_tokens=False)
        prompt_length = len(user_ids)
        input_ids = (user_ids + asm_ids)[:self.max_length]
        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long), 
            "prompt_length": prompt_length,
            "raw_user": ex['user'],
            "raw_assistant": ex['assistant']
        }

print("SFTDataset class defined")

SFTDataset class defined


In [14]:
class SFTDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.pad_id = tokenizer.pad_token_id

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        ex = self.dataset[idx]
        user_ids = self.tokenizer.encode(ex['user'] + self.tokenizer.eos_token, add_special_tokens=False)
        asm_ids = self.tokenizer.encode(ex['assistant'] + self.tokenizer.eos_token, add_special_tokens=False)
        prompt_length = len(user_ids)
        input_ids = (user_ids + asm_ids)[:self.max_length]
        return {"input_ids": torch.tensor(input_ids, dtype=torch.long), "prompt_length": prompt_length}

# -------- SFT Data Collator --------
class SFTDataCollator:
    def __init__(self, tokenizer):
        self.pad_id = tokenizer.pad_token_id

    def __call__(self, features):
        input_ids_list = [ex['input_ids'] for ex in features]
        prompt_lengths = torch.tensor([ex['prompt_length'] for ex in features], dtype=torch.long)
        padded_input_ids = pad_sequence(input_ids_list, batch_first=True, padding_value=self.pad_id)
        return {'input_ids': padded_input_ids, 'prompt_lengths': prompt_lengths}

In [3]:
model="./llada_local"
tokenizer = AutoTokenizer.from_pretrained(model, use_fast=True)
special_tokens_dict = {}
if tokenizer.pad_token is None: special_tokens_dict["pad_token"] = "<|pad|>"
if tokenizer.convert_tokens_to_ids("<|mdm_mask|>") == tokenizer.unk_token_id: special_tokens_dict["additional_special_tokens"] = ["<|mdm_mask|>"]
if special_tokens_dict:
    tokenizer.add_special_tokens(special_tokens_dict)
    model.resize_token_embeddings(len(tokenizer))
mask_id, pad_id = tokenizer.convert_tokens_to_ids("<|mdm_mask|>"), tokenizer.pad_token_id


In [43]:
from torch.utils.data import DataLoader
from datasets import load_from_disk, DatasetDict, concatenate_datasets, interleave_datasets

ds = load_from_disk("./sft_data/combined/")  # Adjust the path to your dataset
train_keys = [k for k in ds.keys() if k.endswith("_train")]
val_keys = [k for k in ds.keys() if k.endswith("_validation")]
print(train_keys)
local_batch_size= 8
train_datasets = [ds[k] for k in train_keys]
interleaved_train = interleave_datasets(train_datasets, probabilities=[0.25,0.25,0.25,0.25], seed=42)
train_ds = SFTDataset(interleaved_train, tokenizer, 8192)
val_ds = SFTDataset(concatenate_datasets([ds[k] for k in ds if k.endswith("_validation")]), tokenizer, 8192)
collator = SFTDataCollator(tokenizer)
train_loader = DataLoader(train_ds, batch_size=local_batch_size, collate_fn=collator)
val_loader = DataLoader(val_ds, batch_size=local_batch_size, collate_fn=collator)

['Llama-Nemotron-Post-Training-Dataset_train', 'Bespoke-Stratos-17k_train', 'OpenThoughts-114k_train', 'databricks-dolly-15k_train', 'SlimOrca_train']


In [44]:
from torch.nn.utils.rnn import pad_sequence
print("Data loaders created successfully")
# please decode input and label tensors to check if they are correct
for batch in train_loader:
    input_ids = batch['input_ids']
    prompt_lengths = batch['prompt_lengths']
    for i in range(len(input_ids)):
        # Decode the full input as a single string
        full_text = tokenizer.decode(input_ids[i], skip_special_tokens=True)
        print(f"Full Input {i}: {full_text}")
        print(f"Prompt Length {i}: {prompt_lengths[i]}")
        
        # Decode just the label part (after prompt_lengths)
        label_tokens = input_ids[i][prompt_lengths[i]:]
        # Remove padding tokens
        label_tokens = label_tokens[label_tokens != tokenizer.pad_token_id]
        label_text = tokenizer.decode(label_tokens, skip_special_tokens=True)
        print(f"Label {i}: {label_text}")
        
        # Also show the prompt part for clarity
        prompt_tokens = input_ids[i][:prompt_lengths[i]]
        prompt_text = tokenizer.decode(prompt_tokens, skip_special_tokens=True)
        print(f"Prompt {i}: {prompt_text}")
        print("-" * 80)
    break

Data loaders created successfully
Full Input 0: conversations: [{'from': 'system', 'value': 'You are a helpful assistant, who always provide explanation. Think like you are answering to a five year old.', 'weight': None}, {'from': 'human', 'value': '2 years…a reality confirmed by a recent ct scan. thank you for all you do.\n\nHow would the previous sentence be correctly capitalized?', 'weight': 0.0}, {'from': 'gpt', 'value': '"2 years... A reality confirmed by a recent CT scan. Thank you for all you do."\n\nIn this sentence, we made sure to capitalize the first letter of each sentence, like "A" in "A reality" and "T" in "Thank you." We also capitalized both "C" and "T" in "CT scan" because it is a short form of "Computed Tomography" and the letters represent the first letter of each word.', 'weight': 1.0}]
Prompt Length 0: 204
Label 0: 
Prompt 0: conversations: [{'from': 'system', 'value': 'You are a helpful assistant, who always provide explanation. Think like you are answering to a f

In [45]:
# Calculate probabilities based on dataset sizes (excluding the last buggy dataset)
print("=== CALCULATING PROBABILITIES BASED ON DATASET SIZES ===")

# Get dataset sizes for training sets
train_sizes = []
for key in train_keys:
    size = len(ds[key])
    train_sizes.append(size)
    print(f"{key}: {size:,} samples")

print(f"\nTotal training datasets: {len(train_keys)}")
print(f"Dataset sizes: {train_sizes}")

# Exclude the last dataset (assuming it's buggy)
if len(train_keys) > 1:
    print(f"\nExcluding last dataset: {train_keys[-1]} (buggy)")
    working_keys = train_keys[:-1]  # All except last
    working_sizes = train_sizes[:-1]  # All except last
else:
    working_keys = train_keys
    working_sizes = train_sizes
    print("\nOnly one dataset, cannot exclude any")

print(f"Working with {len(working_keys)} datasets:")
for key, size in zip(working_keys, working_sizes):
    print(f"  {key}: {size:,} samples")

# Calculate probabilities based on sizes
total_working_samples = sum(working_sizes)
size_based_probabilities = [size / total_working_samples for size in working_sizes]

print(f"\nTotal samples from working datasets: {total_working_samples:,}")
print("\nSize-based probabilities:")
for key, prob in zip(working_keys, size_based_probabilities):
    print(f"  {key}: {prob:.3f} ({prob*100:.1f}%)")

# Create the full probability list (including 0.0 for excluded datasets)
full_probabilities = size_based_probabilities + [0.0] * (len(train_keys) - len(working_keys))
print(f"\nFull probability array for interleave_datasets: {full_probabilities}")

print("\n=== CODE FOR TRAINING SCRIPT ===")
print("# Replace the hard-coded probabilities with this:")
print("train_sizes = [len(ds[k]) for k in train_keys]")
print("working_sizes = train_sizes[:-1]  # Exclude last (buggy) dataset") 
print("total_working = sum(working_sizes)")
print("probabilities = [size/total_working for size in working_sizes] + [0.0]")
print(f"# Result: {full_probabilities}")

# Test the new probabilities
print("\n=== TESTING WITH NEW PROBABILITIES ===")
test_interleaved = interleave_datasets(train_datasets, probabilities=full_probabilities, seed=42)
print(f"Successfully created interleaved dataset with {len(test_interleaved):,} samples")

=== CALCULATING PROBABILITIES BASED ON DATASET SIZES ===
Llama-Nemotron-Post-Training-Dataset_train: 500,000 samples
Bespoke-Stratos-17k_train: 10,345 samples
OpenThoughts-114k_train: 59,988 samples
databricks-dolly-15k_train: 12,007 samples
SlimOrca_train: 414,339 samples

Total training datasets: 5
Dataset sizes: [500000, 10345, 59988, 12007, 414339]

Excluding last dataset: SlimOrca_train (buggy)
Working with 4 datasets:
  Llama-Nemotron-Post-Training-Dataset_train: 500,000 samples
  Bespoke-Stratos-17k_train: 10,345 samples
  OpenThoughts-114k_train: 59,988 samples
  databricks-dolly-15k_train: 12,007 samples

Total samples from working datasets: 582,340

Size-based probabilities:
  Llama-Nemotron-Post-Training-Dataset_train: 0.859 (85.9%)
  Bespoke-Stratos-17k_train: 0.018 (1.8%)
  OpenThoughts-114k_train: 0.103 (10.3%)
  databricks-dolly-15k_train: 0.021 (2.1%)

Full probability array for interleave_datasets: [0.8586049386956074, 0.017764536181612115, 0.1030119861249442, 0.020618

In [46]:
print("\n" + "="*60)
print("CLEAN CODE FOR SFT_TRAIN.PY")
print("="*60)

print("# Training dataset probabilities (size-based, excluding last):")
print("train_sizes = [len(ds[k]) for k in train_keys]")
print("working_sizes = train_sizes[:-1]")
print("total_working = sum(working_sizes)")
print("train_probabilities = [size/total_working for size in working_sizes] + [0.0]")
print("")

print("# Validation dataset probabilities (auto-balanced):")
print("val_probabilities = None")
print("")

print("# Create interleaved datasets:")
print("interleaved_train = interleave_datasets(train_datasets, probabilities=train_probabilities, seed=42)")
print("interleaved_val = interleave_datasets(val_datasets, probabilities=val_probabilities, seed=42)")

print("\n" + "="*60)
print("COPY THIS EXACT CODE BLOCK:")
print("="*60)

code_block = """train_sizes = [len(ds[k]) for k in train_keys]
working_sizes = train_sizes[:-1]
total_working = sum(working_sizes)
train_probabilities = [size/total_working for size in working_sizes] + [0.0]
val_probabilities = None
interleaved_train = interleave_datasets(train_datasets, probabilities=train_probabilities, seed=42)
interleaved_val = interleave_datasets(val_datasets, probabilities=val_probabilities, seed=42)"""

print(code_block)
print("="*60)


CLEAN CODE FOR SFT_TRAIN.PY
# Training dataset probabilities (size-based, excluding last):
train_sizes = [len(ds[k]) for k in train_keys]
working_sizes = train_sizes[:-1]
total_working = sum(working_sizes)
train_probabilities = [size/total_working for size in working_sizes] + [0.0]

# Validation dataset probabilities (auto-balanced):
val_probabilities = None

# Create interleaved datasets:
interleaved_train = interleave_datasets(train_datasets, probabilities=train_probabilities, seed=42)
interleaved_val = interleave_datasets(val_datasets, probabilities=val_probabilities, seed=42)

COPY THIS EXACT CODE BLOCK:
train_sizes = [len(ds[k]) for k in train_keys]
working_sizes = train_sizes[:-1]
total_working = sum(working_sizes)
train_probabilities = [size/total_working for size in working_sizes] + [0.0]
val_probabilities = None
interleaved_train = interleave_datasets(train_datasets, probabilities=train_probabilities, seed=42)
interleaved_val = interleave_datasets(val_datasets, probabilities=

In [7]:
from datasets import interleave_datasets
from torch.utils.data import DataLoader

# EXECUTABLE CODE - Test the size-based approach with dataset size limits
print("=== TESTING SIZE-BASED PROBABILITIES WITH LIMITS ===")
local_batch_size = 8
ds = load_from_disk("./sft_data/combined/")

# Dataset size limits
TRAIN_LIMIT = 50000  # Max 50k samples per training dataset
VAL_LIMIT = 2500     # Max 2.5k samples per validation dataset

train_keys = [k for k in ds.keys() if k.endswith("_train")]
val_keys = [k for k in ds.keys() if k.endswith("_validation")]

# Create limited datasets
limited_train_datasets = []
limited_val_datasets = []
train_sizes = []
val_sizes = []

print("=== TRAINING DATASETS (Limited) ===")
for key in train_keys:
    original_size = len(ds[key])
    limited_size = min(original_size, TRAIN_LIMIT)
    limited_dataset = ds[key].select(range(limited_size))
    limited_train_datasets.append(limited_dataset)
    train_sizes.append(limited_size)
    print(f"{key}: {original_size:,} -> {limited_size:,} samples")

print("\n=== VALIDATION DATASETS (Limited) ===")
for key in val_keys:
    original_size = len(ds[key])
    limited_size = min(original_size, VAL_LIMIT)
    limited_dataset = ds[key].select(range(limited_size))
    limited_val_datasets.append(limited_dataset)
    val_sizes.append(limited_size)
    print(f"{key}: {original_size:,} -> {limited_size:,} samples")

# Calculate probabilities (excluding last training dataset)
working_sizes = train_sizes[:-1] if len(train_sizes) > 1 else train_sizes
working_sizes_val = val_sizes[:-1] if len(val_sizes) > 1 else val_sizes

total_working = sum(working_sizes)
total_working_val = sum(working_sizes_val) if working_sizes_val else 0

train_probabilities = [size/total_working for size in working_sizes] + [0.0] if len(train_sizes) > 1 else [1.0]
val_probabilities = [size/total_working_val for size in working_sizes_val] + [0.0] if len(val_sizes) > 1 else [1.0]

print(f"\nTrain probabilities: {train_probabilities}")
print(f"Val probabilities: {val_probabilities}")

# Create interleaved datasets
interleaved_train_new = interleave_datasets(limited_train_datasets, probabilities=train_probabilities, seed=42)
interleaved_val_new = interleave_datasets(limited_val_datasets, probabilities=val_probabilities, seed=42)

print(f"\nFinal training dataset size: {len(interleaved_train_new):,}")
print(f"Final validation dataset size: {len(interleaved_val_new):,}")

# Create SFT datasets and loaders
collator = SFTDataCollator(tokenizer)
train_ds_new = SFTDataset(interleaved_train_new, tokenizer, 8192)
val_ds_new = SFTDataset(interleaved_val_new, tokenizer, 8192)
train_loader_new = DataLoader(train_ds_new, batch_size=local_batch_size, collate_fn=collator)
val_loader_new = DataLoader(val_ds_new, batch_size=local_batch_size, collate_fn=collator)

print(f"Train loader: {len(train_ds_new):,} samples")
print(f"Val loader: {len(val_ds_new):,} samples")

# Test the new loader
print("\n=== TESTING LIMITED DATASET LOADER ===")
for batch in train_loader_new:
    input_ids = batch['input_ids']
    print(f"Batch shape: {input_ids.shape}")
    full_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    print(f"First example preview: {full_text[:100]}...")
    break

print("\n✅ Limited size-based probability approach working correctly!")

print("\n" + "="*60)
print("CODE FOR SFT_TRAIN.PY WITH LIMITS:")
print("="*60)
code_block = """TRAIN_LIMIT = 50000
VAL_LIMIT = 2500

limited_train_datasets = []
limited_val_datasets = []
train_sizes = []
val_sizes = []

for key in train_keys:
    limited_size = min(len(ds[key]), TRAIN_LIMIT)
    limited_train_datasets.append(ds[key].select(range(limited_size)))
    train_sizes.append(limited_size)

for key in val_keys:
    limited_size = min(len(ds[key]), VAL_LIMIT)
    limited_val_datasets.append(ds[key].select(range(limited_size)))
    val_sizes.append(limited_size)

working_sizes = train_sizes[:-1] if len(train_sizes) > 1 else train_sizes
working_sizes_val = val_sizes[:-1] if len(val_sizes) > 1 else val_sizes
total_working = sum(working_sizes)
total_working_val = sum(working_sizes_val) if working_sizes_val else 0

train_probabilities = [size/total_working for size in working_sizes] + [0.0] if len(train_sizes) > 1 else [1.0]
val_probabilities = [size/total_working_val for size in working_sizes_val] + [0.0] if len(val_sizes) > 1 else [1.0]

interleaved_train = interleave_datasets(limited_train_datasets, probabilities=train_probabilities, seed=42)
interleaved_val = interleave_datasets(limited_val_datasets, probabilities=val_probabilities, seed=42)"""

print(code_block)
print("="*60)

=== TESTING SIZE-BASED PROBABILITIES ===
Dataset sizes: [500000, 10345, 59988, 12007, 414342]
Dataset sizes (validation): [6389973, 2594, 15025, 3003, 103590]
Working sizes (excluding last): [500000, 10345, 59988, 12007]
Train probabilities: [0.8586049386956074, 0.017764536181612115, 0.1030119861249442, 0.020618538997836316, 0.0]
Val probabilities: [0.9967831379146553, 0.0004046426267764537, 0.002343776201741024, 0.0004684432568271744, 0.0]
Total working samples: 582340
Total working validation samples: 6410595

New training dataset size: 580,006
New validation dataset size: 6,403,591


NameError: name 'collator' is not defined